<a href="https://colab.research.google.com/github/HanlynnKe/nlp-demo/blob/master/chinese_text_binary_classification_with_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install bert-tensorflow

     |████████████████████████████████| 71kB 2.4MB/s 


In [2]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import pickle
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
from bert import modeling

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

!mkdir model
!ls

!unzip -d 'model/' '/content/drive/My Drive/chinese_wwm_ext_L-12_H-768_A-12.zip'
!ls model

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/
drive  model  sample_data
Archive:  /content/drive/My Drive/chinese_wwm_ext_L-12_H-768_A-12.zip
  inflating: model/bert_config.json  
  inflating: model/bert_model.ckpt.data-00000-of-00001  
  inflating: model/bert_model.ckpt.index  
  inflating: model/bert_model.ckpt.meta  
  inflating: model/vocab.txt         
bert_config.json		     bert_model.ckpt.index  vocab.txt
bert_model.ckpt.data-00000-of-00001  bert_model.ckpt.meta


In [0]:
def pretty_print(result):
    df = pd.DataFrame([result]).T
    df.columns = ["values"]
    return df

In [0]:
def make_features(dataset, label_list, MAX_SEQ_LENGTH, 
                  tokenizer, DATA_COLUMN, LABEL_COLUMN):
    input_example = dataset.apply(lambda x: bert.run_classifier. InputExample(guid=None,
                                                   text_a = x[DATA_COLUMN],
                                                   text_b = None,
                                                   label = x[LABEL_COLUMN]),
                                                   axis = 1)
    features = bert.run_classifier.convert_examples_to_features(input_example, label_list, MAX_SEQ_LENGTH, tokenizer)
    return features

def create_model(bert_config, is_predicting,
                 input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""
  is_training = not is_predicting

  model = bert.modeling.BertModel(
      config=bert_config,
      is_training=is_training,
      input_ids=input_ids,
      input_mask=input_mask,
      token_type_ids=segment_ids)
      # use_one_hot_embeddings=use_one_hot_embeddings)
  
  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = model.get_pooled_output()

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    regularizer = tf.nn.l2_loss(output_weights)
    loss = tf.reduce_mean(loss + 0.0001 * regularizer)
    return (loss, predicted_labels, log_probs)

# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(bert_config, init_checkpoint, num_labels, 
                     learning_rate, num_train_steps, num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]
      
    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    is_training = (mode == tf.estimator.ModeKeys.TRAIN)
    is_eval = (mode == tf.estimator.ModeKeys.EVAL)

    tvars = tf.trainable_variables()

    initialized_variable_names = {}
    (assignment_map, initialized_variable_names) = \
        modeling.get_assignment_map_from_checkpoint(tvars, init_checkpoint)
      
    tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
    
    # TRAIN and EVAL
    if is_training or is_eval:

      (loss, predicted_labels, log_probs) = create_model(
        bert_config, is_predicting, input_ids, input_mask, 
          segment_ids, label_ids, num_labels)
    

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        bert_config, is_predicting, input_ids, 
          input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn

def estimator_builder(bert_config, init_checkpoint, OUTPUT_DIR, 
                      SAVE_SUMMARY_STEPS, SAVE_CHECKPOINTS_STEPS, label_list, 
                      LEARNING_RATE, num_train_steps, num_warmup_steps, 
                      BATCH_SIZE):

    # Specify outpit directory and number of checkpoint steps to save
    run_config = tf.estimator.RunConfig(
        model_dir=OUTPUT_DIR,
        save_summary_steps=SAVE_SUMMARY_STEPS,
        save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

    model_fn = model_fn_builder(
      bert_config = bert_config, 
      init_checkpoint = init_checkpoint,
      num_labels=len(label_list),
      learning_rate=LEARNING_RATE,
      num_train_steps=num_train_steps,
      num_warmup_steps=num_warmup_steps)

    estimator = tf.estimator.Estimator(
      model_fn=model_fn,
      config=run_config,
      params={"batch_size": BATCH_SIZE})
    return estimator, model_fn, run_config


In [0]:
def run_on_dfs(train, test, DATA_COLUMN, LABEL_COLUMN, 
               MAX_SEQ_LENGTH = 128,
               BATCH_SIZE = 8,
               LEARNING_RATE = 3e-5,
               NUM_TRAIN_EPOCHS = 3.0,
               WARMUP_PROPORTION = 0.1,
               SAVE_SUMMARY_STEPS = 100,
               SAVE_CHECKPOINTS_STEPS = 10000,
               vocab_file = "model/vocab.txt",
               bert_config_file = "model/bert_config.json",
               init_checkpoint = "model/bert_model.ckpt"):

    label_list = train[LABEL_COLUMN].unique().tolist()
    
    tokenizer = tokenization.FullTokenizer(vocab_file=vocab_file, do_lower_case=False)

    train_features = make_features(train, label_list, MAX_SEQ_LENGTH, tokenizer, DATA_COLUMN, LABEL_COLUMN)
    test_features = make_features(test, label_list, MAX_SEQ_LENGTH, tokenizer, DATA_COLUMN, LABEL_COLUMN)

    num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
    num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

    bert_config = modeling.BertConfig.from_json_file(bert_config_file)

    estimator, model_fn, run_config = estimator_builder(
                                  bert_config,
                                  init_checkpoint,
                                  OUTPUT_DIR,
                                  SAVE_SUMMARY_STEPS,
                                  SAVE_CHECKPOINTS_STEPS,
                                  label_list,
                                  LEARNING_RATE,
                                  num_train_steps,
                                  num_warmup_steps,
                                  BATCH_SIZE)

    train_input_fn = bert.run_classifier.input_fn_builder(
        features=train_features,
        seq_length=MAX_SEQ_LENGTH,
        is_training=True,
        drop_remainder=False)

    estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)

    test_input_fn = run_classifier.input_fn_builder(
        features=test_features,
        seq_length=MAX_SEQ_LENGTH,
        is_training=False,
        drop_remainder=False)

    result_dict = estimator.evaluate(input_fn=test_input_fn, steps=None)
    return result_dict, estimator
    

In [0]:
import random
random.seed(0)
OUTPUT_DIR = 'output'

----- you just need to focus from here ------

## Get your data

In [8]:
!wget https://github.com/HanlynnKe/nlp-demo/raw/master/safety_train_valid.pickle
!wget https://github.com/HanlynnKe/nlp-demo/raw/master/test_new.csv

--2019-11-26 02:00:34--  https://github.com/HanlynnKe/nlp-demo/raw/master/safety_train_valid.pickle
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/HanlynnKe/nlp-demo/master/safety_train_valid.pickle [following]
--2019-11-26 02:00:35--  https://raw.githubusercontent.com/HanlynnKe/nlp-demo/master/safety_train_valid.pickle
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1181866 (1.1M) [application/octet-stream]
Saving to: ‘safety_train_valid.pickle’

safety_train_valid. 100%[===================>]   1.13M  --.-KB/s    in 0.09s   

2019-11-26 02:00:36 (12.9 MB/s) - ‘safety_train_valid.pickle’ saved [1181

In [9]:
with open("safety_train_valid.pickle", 'rb') as f:
    train, test = pickle.load(f)

train = train.sample(len(train))
test = test.sample(len(test))
train.head()

,label,comment
284,0,东西很好吃、服务员很漂亮
6475,0,尖椒兔全是骨头肉少得可怜！ 差评！！！
2482,0,第二次来了，还是挺实惠的，分量也可以！
1676,1,上班每天中午都点外卖，昨天中午吃了你家的东西，晚上没吃，今天一直拉肚子。
8964,0,我想说的是一点都不好吃，感觉汤都很混那种，也没有什么味道


In [0]:
myparam = {
        "DATA_COLUMN": "comment",
        "LABEL_COLUMN": "label",
        "BATCH_SIZE": 8,
        "LEARNING_RATE": 3e-5, #6e-5 1e-4 2e-5
        "NUM_TRAIN_EPOCHS":4,
        "vocab_file": "model/vocab.txt",
        "bert_config_file": "model/bert_config.json",
        "init_checkpoint": "model/bert_model.ckpt"
    }

In [13]:
result, estimator = run_on_dfs(train, test, **myparam)

INFO:tensorflow:Writing example 0 of 8000
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: None
INFO:tensorflow:tokens: [CLS] 东 西 很 好 吃 、 服 务 员 很 漂 亮 [SEP]
INFO:tensorflow:input_ids: 101 691 6205 2523 1962 1391 510 3302 1218 1447 2523 4023 778 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [14]:
pretty_print(result)

,values
auc,0.903247
eval_accuracy,0.956500
f1_score,0.842676
false_negatives,48.000000
false_positives,39.000000
loss,0.206666
precision,0.856618
recall,0.829181
true_negatives,1680.000000
true_positives,233.000000


In [0]:
def getPrediction(in_sentences, estimator):
  labels = ["Negaitive", "Positive"]
  label_list = train['label'].unique().tolist()
  MAX_SEQ_LENGTH = 128
  vocab_file = "model/vocab.txt"
  bert_config_file = "model/bert_config.json"
  init_checkpoint = "model/bert_model.ckpt"
  
  tokenizer = tokenization.FullTokenizer(vocab_file=vocab_file, do_lower_case=False)
  input_examples = [bert.run_classifier.InputExample(guid=None, 
                                                     text_a=x, 
                                                     text_b=None, 
                                                     label=0) 
                    for x in in_sentences]
  input_features = bert.run_classifier.convert_examples_to_features(input_examples, 
                                                                    label_list, 
                                                                    MAX_SEQ_LENGTH, 
                                                                    tokenizer)
  predict_input_fn = bert.run_classifier.input_fn_builder(
      features=input_features, 
      seq_length=MAX_SEQ_LENGTH, 
      is_training=False, 
      drop_remainder=False)

  predictions = estimator.predict(predict_input_fn)
  
  return [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]

In [16]:
target = pd.read_csv('test_new.csv')
pred_sentences = target['comment'].values.tolist()
predictions = getPrediction(pred_sentences, estimator)

INFO:tensorflow:Writing example 0 of 2000
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: None
INFO:tensorflow:tokens: [CLS] 糯 米 团 是 我 小 时 候 的 记 忆 了 ， 吃 起 还 是 好 吃 ， 只 是 小 时 候 的 油 条 没 有 这 么 硬 ！ 油 茶 也 还 好 ！ 可 以 试 试 [SEP]
INFO:tensorflow:input_ids: 101 5140 5101 1730 3221 2769 2207 3198 952 4638 6381 2554 749 8024 1391 6629 6820 3221 1962 1391 8024 1372 3221 2207 3198 952 4638 3779 3340 3766 3300 6821 720 4801 8013 3779 5763 738 6820 1962 8013 1377 809 6407 6407 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [17]:
predictions

[('糯米团是我小时候的记忆了，吃起还是好吃，只是小时候的油条没有这么硬！油茶也还好！可以试试',
  array([-3.3087024e-04, -8.0141001e+00], dtype=float32),
  'Negaitive'),
 ('满满的五星好评，口味好，服务好，特别喜欢，昨天第一次买，今天就回购了，买的刨奶，店长问我加腰果还是核桃，我说随便，他又问我喜欢吃什么，我说都喜欢，然后，帅帅的店长都给我加了，超赞',
  array([-4.4395603e-04, -7.7199893e+00], dtype=float32),
  'Negaitive'),
 ('好喝！经常会再去买来喝！就是排队的人太多了',
  array([-3.294402e-04, -8.018146e+00], dtype=float32),
  'Negaitive'),
 ('三个人订的四人餐，菜量大没吃完，问道不错。',
  array([-3.9259833e-04, -7.8428359e+00], dtype=float32),
  'Negaitive'),
 ('好的一如既往，真真爱上了自助炒饭自助八宝粥自助冰粉！！！喜欢所有菜和肉，两女一男吃两份两人餐没吃完，饮料也不错！有空就要来！',
  array([-4.094001e-04, -7.801081e+00], dtype=float32),
  'Negaitive'),
 ('太tmd的难吃了跟狗屎一样草',
  array([-7.5943238e-04, -7.1833224e+00], dtype=float32),
  'Negaitive'),
 ('火锅味的味道比渝李记那个火锅米线的味道好吃，更有一种老火锅的感觉，很划算，但是有韭菜叶的就好了，更喜欢吃韭菜叶的面',
  array([-4.2274597e-04, -7.7688642e+00], dtype=float32),
  'Negaitive'),
 ('今天肯定是走着来的，饿坏宝宝了',
  array([-0.00525806, -5.2506194 ], dtype=float32),
  'Negaitive'),
 ('差评。  饭是馊的！根本吃不下，再也不会点了！菜没问题，但是不可能不吃饭啊，被迫去买了方

In [0]:
target['label'] = None
for row in range(len(predictions)):
    if predictions[row][2] == 'Positive':
        target.loc[row, 'label'] = 1
    else:
        target.loc[row, 'label'] = 0    

In [19]:
target.head()

,id,comment,label
0,0011f384-9e54-4fb4-a272-330a6cab6804,糯米团是我小时候的记忆了，吃起还是好吃，只是小时候的油条没有这么硬！油茶也还好！可以试试,0
1,00223e4f-47e1-4fc8-9657-06444a7de9a5,满满的五星好评，口味好，服务好，特别喜欢，昨天第一次买，今天就回购了，买的刨奶，店长问我加腰...,0
2,00225350-c169-435c-84cf-970068df5b12,好喝！经常会再去买来喝！就是排队的人太多了,0
3,00a3190c-90c1-44c3-b809-7a9b1314cd27,三个人订的四人餐，菜量大没吃完，问道不错。,0
4,00b3f76e-fda3-42cd-8884-25e03a5dba64,好的一如既往，真真爱上了自助炒饭自助八宝粥自助冰粉！！！喜欢所有菜和肉，两女一男吃两份两人餐...,0


In [0]:
target_o = target[['id', 'label']]
target_o.to_csv('output_BERT.csv', index=False)

In [0]:
from google.colab import files
files.download('output_BERT.csv') 